# Week 4: Text and dates

1. Text
    - Working with text data via the `str` accessor
    - Using `str` to clean integer data
    - Getting textual statistics 
    - Cleaning text + strings
2. Dates and times
    - How do dates and times work as data structures?
    - `datetime` and `timedelta` objects
    - Reading date information from CSV files
    - Retrieving via dates and times
    - Time series -- setting the index to use a datetime column
    - Resampling -- grouping via time

In [1]:
import numpy as np
import pandas as pd
from pandas import Series, DataFrame

In [2]:
# load NYC taxi data from January, 2019
filename = '/Users/reuven/Courses/Current/data/nyc_taxi_2019-01.csv'

df = pd.read_csv(filename, 
                usecols=['passenger_count', 'trip_distance', 'total_amount'])

In [3]:
df.head()

,passenger_count,trip_distance,total_amount
0,1,1.5,9.95
1,1,2.6,16.30
2,3,0.0,5.80
3,5,0.0,7.55
4,5,0.0,55.55


In [4]:
df.dtypes

passenger_count      int64
trip_distance      float64
total_amount       float64
dtype: object

In [6]:
# what if I want to find the 10 shortest-distance trips?

df.sort_values(by='trip_distance').head(10)

,passenger_count,trip_distance,total_amount
7667791,1,0.0,0.00
4863796,0,0.0,3.30
4863795,2,0.0,3.30
4863794,2,0.0,3.30
4863793,1,0.0,3.30
4863792,1,0.0,3.30
4863789,1,0.0,5.30
4863768,1,0.0,3.30
4863743,1,0.0,3.96
2682283,1,0.0,20.30


In [ ]:
# what if I want to sort first by trip_distance, and then (in the case of a tie) by total_amount?
